In [52]:
from dotenv import load_dotenv
load_dotenv()

True

In [53]:
import os
from glob import glob

from pprint import pprint
import json

In [54]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-m3', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [55]:
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_sample_collection",     #db에 있는 테이블
    persist_directory="./chroma_db",            #로컬 디렉토리에 어떻게 넣을거냐
    embedding_function=embeddings_model,
)

In [56]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 한 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [57]:
from langchain_core.documents import Document


#5개의 도큐먼트 객체 만들어서 크로마디비안에 저장 
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",   #content
    "머신러닝은 인공지능의 한 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.",
]

doc_objects = []

for idx, content in enumerate(documents, start=1):
    doc = Document(
        page_content=content, 
        metadata={'source': "AI_Textbook", "chapter": f"Chapter {idx}"},
    )
    doc_objects.append(doc)
    
# 순차적 ID 리스트 생성
doc_ids = [f"doc_{i}" for i in range(1, len(doc_objects) + 1)]

added_docs_ids = chroma_db.add_documents(documents=doc_objects, ids=doc_ids)

print(f'{len(added_docs_ids)}개의 문서가 성공적으로 DB에 추가되었습니다.')
print(added_docs_ids)

5개의 문서가 성공적으로 DB에 추가되었습니다.
['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5']


In [58]:
query = "인공지능과 머신러닝의 관계는?"
#embedding_function=embeddings_model, 이렇게 임베딩모델객체로 넣어줘서 따로 임베딩 안해도 됨. 
results = chroma_db.similarity_search(query, k=2)

print(f"질문: {query}")
for doc in results:
    print(f"문서 ID: {doc.metadata['chapter']}, 내용: {doc.page_content}")

질문: 인공지능과 머신러닝의 관계는?
문서 ID: Chapter 2, 내용: 머신러닝은 인공지능의 한 분야입니다.
문서 ID: Chapter 3, 내용: 딥러닝은 머신러닝의 한 종류입니다.


In [59]:
#chroma_db.update_documents(document_ids=added_docs_ids, document=doc_objects)
#Document(page_content, metadata={})


In [60]:
updated_content = [
    "신경망은 인간의 뇌 구조를 모방한 딥러닝의 핵심 구조입니다.",
    "빅데이터는 머신러닝 모델 학습에 필요한 대용량의 데이터를 의미합니다."
]

updated_docs = []
start_chapter = len(documents) + 1  # 6부터 시작

for idx, content in enumerate(updated_content, start=start_chapter):
    doc = Document(
        page_content=content,
        metadata={"source": "AI_Textbook", "chapter": f"Chapter {idx}"}
    )
    updated_docs.append(doc)

In [61]:
print(f"질문: {query}")
for doc in results:
    print(f"문서 ID: {doc.metadata['chapter']}, 내용: {doc.page_content}")

질문: 인공지능과 머신러닝의 관계는?
문서 ID: Chapter 2, 내용: 머신러닝은 인공지능의 한 분야입니다.
문서 ID: Chapter 3, 내용: 딥러닝은 머신러닝의 한 종류입니다.


In [62]:
update_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포합합니다.",
    metadata={"source": "AI_Textbook", "chapter": "Chapter 6"},
)

chroma_db.update_document(document_id='doc_1', document=update_document_1)

print('문서 업데이트 완료')

문서 업데이트 완료


In [63]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포합합니다.',
  '머신러닝은 인공지능의 한 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [64]:
chroma_db.delete(ids=['doc_5'])

In [65]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포합합니다.',
  '머신러닝은 인공지능의 한 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [66]:
chroma_db2 = Chroma(
    collection_name="ai_sample_collection",
    persist_directory="./chroma_db",
    embedding_function=embeddings_model,
)

chroma_db2.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포합합니다.',
  '머신러닝은 인공지능의 한 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [67]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from transformers import AutoTokenizer

In [69]:
#데이터 로드
def load_text_files(text_files):
    data = []
    for file in text_files:
        loader = TextLoader(file, encoding='utf-8')
        data += loader.load()
        
    return data

korean_text_files = glob(os.path.join("data", "*_KR.txt"))
korean_data = load_text_files(korean_text_files)
# korean_data

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,            #bge-m3의 기준으로 알아서 잘 잘라줌. 
    separator=r"[.!?]\s+",
    chunk_size=100,
    chunk_overlap=0,
    is_separator_regex=True,
    keep_separator=True,
)

korean_docs = text_splitter.split_documents(korean_data)

print("한국어 문서 수: ", len(korean_docs))

한국어 문서 수:  8


In [ ]:

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embeddings_huggingface = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

chroma_db = Chroma.from_documents(
    documents=korean_docs,
    embedding=embeddings_huggingface,
    collection_name="db_korean_cosine",
    persist_directory="./chroma_db",
    collection_metadata={"hnsw:space" : "cosine"},      #원래 유클리드 자리 (12, ip, cosine 등)
    #임베디드 된 문서마다 다르기 때문에 상황에 따라 맞는거 사용
    
)


In [71]:
chroma_db.get()

{'ids': ['7419c14c-a63e-40ca-a06a-515bb493c2da',
  '978fa4f8-cf5f-4f9e-a7cf-b1e12e0a3f54',
  '99547b22-d522-48d3-916a-fc5e1c11c0ae',
  'd287c92c-ba67-4a22-8c58-f3f2cd7468a8',
  '55db9d38-a5ff-4690-8826-06efadd09abc',
  'a70ea8a7-090c-49eb-bcfb-545cda64f294',
  '7bf8b05d-6e46-49f1-92c2-8619029a0276',
  'ab7e4a4b-3e82-426d-9f7f-9a915e546c30'],
 'embeddings': None,
 'documents': ['리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다',
  '.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다',
  '. 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다',
  '.\n\n리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.',
  '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크

In [72]:
#sementic search
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k":2},
    
)
query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색결과")
for doc in retrieved_docs:
    print(f" - {doc.page_content} [출처 : {doc.metadata['source']}]")   



쿼리: 리비안은 언제 사업을 시작했나요?
검색결과
 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처 : data\리비안_KR.txt]
 - .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처 : data\리비안_KR.txt]


### MMR (Maximal Margimal Relevance) 검색

In [100]:
from re import search
from langchain_community.utils.math import  cosine_similarity

chroma_mma = chroma_db.as_retriever(
    search_type = "mmr",
    search_kwargs = {
        "k" : 2,
        "fetch_k" : 8,      #MMR을 위한 후보 문서 수
        "lamda_mult" : 0.5,     #MMR의 다양성 조절 파라미터 ( 1으 최소 다양성, 0은 최대 다양성)
    },
)

query = "리비안은 어떤 회사인가요?"
retrieved_docs = chroma_mma.invoke(query)

print(f"쿼리: {query}")
print(f"검색결과")

for doc in retrieved_docs: 
    score = cosine_similarity(
        [ embeddings_huggingface.embed_query(query)],
        [ embeddings_huggingface.embed_query(doc.page_content)]
    )[0][0]
    print(f" - {doc.page_content}  (유사도: {score: .4f})")

쿼리: 리비안은 어떤 회사인가요?
검색결과
 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다  (유사도:  0.6815)
 - .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다  (유사도:  0.5930)


In [101]:
from re import search
from langchain_community.utils.math import  cosine_similarity

chroma_mma = chroma_db.as_retriever(
    search_type = "mmr",
    search_kwargs = {
        "k" : 3,
        "fetch_k" : 8,      #MMR을 위한 후보 문서 수
        "lamda_mult" : 0.9,     #MMR의 다양성 조절 파라미터 ( 1은 최소 다양성, 0은 최대 다양성)
    },
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_mma.invoke(query)

print(f"쿼리: {query}")
print(f"검색결과")

for doc in retrieved_docs: 
    score = cosine_similarity(
        [ embeddings_huggingface.embed_query(query)],
        [ embeddings_huggingface.embed_query(doc.page_content)]
    )[0][0]
    print(f" - {doc.page_content}  (유사도: {score: .4f})")

쿼리: 리비안은 언제 사업을 시작했나요?
검색결과
 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다  (유사도:  0.6734)
 - .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다  (유사도:  0.5955)
 - .

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.  (유사도:  0.5690)


In [ ]:
chroma_mma = chroma_db.as_retriever(
    search_type = "mmr",
    search_kwargs = {
        "k" : 2,
        "fetch_k" : 8,      #MMR을 위한 후보 문서 수
        "lamda_mult" : 0.9,     #MMR의 다양성 조절 파라미터 ( 1은 최소 다양성, 0은 최대 다양성)
        "score_threshold" : 0.1,    #유사도 점수 임계값
    },
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_mma.invoke(query)

for doc in retrieved_docs: 
    score = cosine_similarity(
        [ embeddings_huggingface.embed_query(query)],
        [ embeddings_huggingface.embed_query(doc.page_content)]
    )[0][0]
    print(f" - {doc.page_content}  (유사도: {score: .4f})")

 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다  (유사도:  0.6734)
 - .

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.  (유사도:  0.5690)


#### Similary score threshold (기준 스코어 이상인 문서를 대상으로 추출) 검색

In [111]:
chroma_mma = chroma_db.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs = {
        "k" : 3,
        "score_threshold" : 0.6,    #유사도 점수 임계값
    },
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_mma.invoke(query)

for doc in retrieved_docs: 
    score = cosine_similarity(
        [ embeddings_huggingface.embed_query(query)],
        [ embeddings_huggingface.embed_query(doc.page_content)]
    )[0][0]
    print(f" - {doc.page_content}  (유사도: {score: .4f})")

 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다  (유사도:  0.6734)


#### metadata

In [113]:
chroma_metadata = chroma_db.as_retriever(
    search_kwargs = {
        "k" : 8,        #검색할 문서 수 
        "filter" : {'source' : "data\리비안_KR.txt"}    #메타데이터 필터링
    }
)


query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_metadata.invoke(query)

print(f"쿼리: {query}")
print(f"검색결과")

for doc in retrieved_docs:
    print(f" - {doc.page_content}  (유사도: {score: .4f}) ")

쿼리: 리비안은 언제 사업을 시작했나요?
검색결과
 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다  (유사도:  0.6734) 
 - .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다  (유사도:  0.6734) 
 - .

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.  (유사도:  0.6734) 
 - . 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다  (유사도:  0.6734) 


In [116]:
chroma_content = chroma_db.as_retriever(
    search_kwargs = {
        "k" : 2,        #검색할 문서 수 
        "where_document" : {"$contains" : "리비안"}    #문서 내용 필터링 
    }
)


query = "언제 사업을 시작했나요?"
retrieved_docs = chroma_content.invoke(query)

print(f"쿼리: {query}")
print(f"검색결과")

for doc in retrieved_docs:
    print(f" - {doc.page_content}  [출처 : {doc.metadata['source']}] ")

쿼리: 언제 사업을 시작했나요?
검색결과
 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다  [출처 : data\리비안_KR.txt] 
 - .

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.  [출처 : data\리비안_KR.txt] 


In [117]:
chroma_content = chroma_db.as_retriever(
    search_kwargs = {
        "k" : 2,        #검색할 문서 수 
        "where_document" : {"$contains" : "테슬라"}    #문서 내용 필터링 
    }
)


query = "언제 사업을 시작했나요?"
retrieved_docs = chroma_content.invoke(query)

print(f"쿼리: {query}")
print(f"검색결과")

for doc in retrieved_docs:
    print(f" - {doc.page_content}  [출처 : {doc.metadata['source']}] ")

쿼리: 언제 사업을 시작했나요?
검색결과
 - 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다  [출처 : data\테슬라_KR.txt] 
 - . 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다  [출처 : data\테슬라_KR.txt] 


In [121]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",  # OpenAI의 GPT-4 모델
    temperature=0.0,  # 생성되는 응답의 다양성 조절 ; 창의서 1도 없는 상태
    max_tokens=100,  # 최대 토큰 수
)

response = llm.invoke(
    "테슬라 창업자는 누구인가요?")
print(f"LLM 응답 : {response}")  # LLM의 응답 내용 출력

LLM 응답 : content='테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하고 회사의 비전을 이끌어가면서 테슬라의 주요 인물로 자리잡게 되었습니다' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 17, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'length', 'logprobs': None} id='run-e135d929-e2a8-41b0-9a0d-79934d32231c-0' usage_metadata={'input_tokens': 17, 'output_tokens': 100, 'total_tokens': 117}


In [127]:
#context를 제공하고 답변 생성

query_with_context = f"""{retrieved_docs[0].page_content}
\n\n위 내용에 근거하여 다음 질문에 답변하세요. \n\n{query}"""

print(query_with_context)

테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다


위 내용에 근거하여 다음 질문에 답변하세요. 

테슬라 창업자는 누구인가요?


In [128]:
response = llm.invoke(query_with_context)

print(f"LLM 응답 : {response}")

LLM 응답 : content='테슬라의 창업자는 마틴 에버하드와 마크 타페닝입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 124, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None} id='run-ca23c89d-0094-4399-aa47-5992e4c5737f-0' usage_metadata={'input_tokens': 124, 'output_tokens': 21, 'total_tokens': 145}


In [129]:
### 전체 RAG 파이프라인 구성

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

#llm 초기화

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0, max_tokens=100)

# 프롬프트 생성

retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면, 
'주어진 정보로는 답변할 수 없습니다.' 라고 답변해주세요.

컨텍스트 : {context}
질문 : {input}
답변 : 
""")


#체인 생성

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

rag_chain = create_retrieval_chain(chroma_content, combine_docs_chain)

# RAG 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke({"input": query}) #전부 프롬프트에 모이는거야. 
print(response)  # RAG 체인의 응답 내용 출력

{'input': '테슬라 창업자는 누구인가요?', 'context': [Document(metadata={'source': 'data\\테슬라_KR.txt'}, page_content='테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다'), Document(metadata={'source': 'data\\테슬라_KR.txt'}, page_content='. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다')], 'answer': '테슬라의 창업자는 마틴 에버하드와 마크 타페닝입니다.'}


In [130]:
### Gradio 챗봇

import gradio as gr
def answer_invoke(message, history):
    response = rag_chain.invoke({"input": message})
    return response['answer']

demo = gr.ChatInterface(fn=answer_invoke, title="miny bot")

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\KOSA\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-e_qH7Re7-py3.11\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [147]:
demo.close()

Closing server running on port: 7865


In [148]:
retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

In [149]:
# RAG 체인 생성
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context.
Do not use any external information or knowledge.
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question]
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)


# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [150]:
retriever_chain = retriever | format_docs

In [151]:
# LLM 체인 생성
rag_chain = (
    {"context": retriever_chain, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
import gradio as gr
from langchain_core.messages import HumanMessage, AIMessage

def answer_invoke(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    #키보드에 입력한 메시지 값
    history_langchain_format.append(HumanMessage(content=message))
        
    
    response = rag_chain.invoke(message)
    
    final_answer = llm.invoke(
       history_langchain_format[:-1] + [AIMessage(content=response)] +
       [HumanMessage(content=message)], 
       #과거 질의 + ai대답 + 방금 한 질문 
    )
    return final_answer.content

demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

demo.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


c:\Users\KOSA\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-e_qH7Re7-py3.11\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
